In [1]:
import requests
import pandas as pd

import datetime
import reverse_geocoder as rg

https://public.emdat.be/data <br>
datos sobre fallecidos por año en todos los países

In [2]:
df = pd.read_excel('../../DataSet/raw/emdat_public_2023_07_05_query_uid-U68zMR.xlsx',skiprows=6)
df

c:\Users\sgraf\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,Dis No,Year,Seq,Glide,Disaster Group,Disaster Subgroup,Disaster Type,Disaster Subtype,Disaster Subsubtype,Event Name,...,"Reconstruction Costs, Adjusted ('000 US$)",Insured Damages ('000 US$),"Insured Damages, Adjusted ('000 US$)",Total Damages ('000 US$),"Total Damages, Adjusted ('000 US$)",CPI,Adm Level,Admin1 Code,Admin2 Code,Geo Locations
0,1906-0002-COL,1906,2,NaN,Natural,Geophysical,Earthquake,Ground movement,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.077011,NaN,NaN,NaN,NaN
1,1950-0028-COL,1950,28,NaN,Natural,Geophysical,Earthquake,Ground movement,NaN,NaN,...,NaN,NaN,NaN,5000.0,60683.0,8.239552,NaN,NaN,NaN,NaN
2,1962-0030-COL,1962,30,NaN,Natural,Geophysical,Earthquake,Ground movement,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,10.336410,NaN,NaN,NaN,NaN
3,1967-0020-COL,1967,20,NaN,Natural,Geophysical,Earthquake,Ground movement,NaN,NaN,...,NaN,NaN,NaN,600.0,5264.0,11.398527,NaN,NaN,NaN,NaN
4,1970-0051-COL,1970,51,NaN,Natural,Geophysical,Earthquake,Ground movement,NaN,NaN,...,NaN,NaN,NaN,400.0,3015.0,13.266480,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141,1973-0148-USA,1973,148,NaN,Natural,Geophysical,Earthquake,Ground movement,NaN,NaN,...,NaN,NaN,NaN,1000.0,6591.0,15.171453,NaN,NaN,NaN,NaN
142,1973-0149-USA,1973,149,NaN,Natural,Geophysical,Earthquake,Ground movement,NaN,NaN,...,NaN,NaN,NaN,5750.0,37900.0,15.171453,NaN,NaN,NaN,NaN
143,1975-0155-USA,1975,155,NaN,Natural,Geophysical,Earthquake,Ground movement,NaN,NaN,...,NaN,NaN,NaN,1000.0,5438.0,18.389129,NaN,NaN,NaN,NaN
144,2022-0808-USA,2022,808,NaN,Natural,Geophysical,Earthquake,Ground movement,NaN,NaN,...,NaN,NaN,NaN,250000.0,250000.0,100.000000,2,NaN,28876,Humboldt (Adm2).


In [3]:
df.columns

Index(['Dis No', 'Year', 'Seq', 'Glide', 'Disaster Group', 'Disaster Subgroup',
       'Disaster Type', 'Disaster Subtype', 'Disaster Subsubtype',
       'Event Name', 'Country', 'ISO', 'Region', 'Continent', 'Location',
       'Origin', 'Associated Dis', 'Associated Dis2', 'OFDA Response',
       'Appeal', 'Declaration', 'AID Contribution ('000 US$)', 'Dis Mag Value',
       'Dis Mag Scale', 'Latitude', 'Longitude', 'Local Time', 'River Basin',
       'Start Year', 'Start Month', 'Start Day', 'End Year', 'End Month',
       'End Day', 'Total Deaths', 'No Injured', 'No Affected', 'No Homeless',
       'Total Affected', 'Reconstruction Costs ('000 US$)',
       'Reconstruction Costs, Adjusted ('000 US$)',
       'Insured Damages ('000 US$)', 'Insured Damages, Adjusted ('000 US$)',
       'Total Damages ('000 US$)', 'Total Damages, Adjusted ('000 US$)', 'CPI',
       'Adm Level', 'Admin1 Code', 'Admin2 Code', 'Geo Locations'],
      dtype='object')

dejamos las columnas que nos interesan

In [4]:
df = df[['Year', 'Disaster Subsubtype',
'ISO', 'Location',
'Origin',
'Dis Mag Value',
'Latitude', 'Longitude', 'Local Time',
'Start Year', 'Start Month', 'Start Day', 'End Year', 'End Month', 'End Day', 
'Total Deaths', 'No Injured', 'No Affected', 'No Homeless',
'Total Affected',
"Insured Damages ('000 US$)", "Insured Damages, Adjusted ('000 US$)",
"Total Damages ('000 US$)", "Total Damages, Adjusted ('000 US$)", 'CPI']].copy()

In [5]:
df.rename(columns={"Insured Damages ('000 US$)":"Insured Damages US$", "Insured Damages, Adjusted ('000 US$)": "Insured Damages US$(2021)",
                   "Total Damages ('000 US$)":"Total Damages US$", "Total Damages, Adjusted ('000 US$)": "Total Damages US$(2021)"}, inplace=True)

Juntar columnas de fecha

In [6]:
df['Start Year'] = df['Start Year'].astype(str)
df['Start Month'] = df['Start Month'].astype(str)
df['Start Day'] = df['Start Day'].astype(str)
df['Start Year'] = df[['Start Year', 'Start Month', 'Start Day']].agg('-'.join, axis=1).apply(lambda x: pd.to_datetime(x, format='%Y/%m/%d'))
df.rename(columns={'Start Year':'date'}, inplace=True)
df.drop(columns=['Start Day', 'Start Month'],inplace=True)

df['End Year'] = df['End Year'].astype(str)
df['End Month'] = df['End Month'].astype(str)
df['End Day'] = df['End Day'].astype(str)
df['End Year'] = df[['End Year', 'End Month', 'End Day']].agg('-'.join, axis=1).apply(lambda x: pd.to_datetime(x, format='%Y-%m-%d'))
df.rename(columns={'End Year':'End date'},inplace=True)
df.drop(columns=['End Day', 'End Month'],inplace=True)

In [7]:
df['Total Damages US$']

0           NaN
1        5000.0
2           NaN
3         600.0
4         400.0
         ...   
141      1000.0
142      5750.0
143      1000.0
144    250000.0
145         NaN
Name: Total Damages US$, Length: 146, dtype: float64

In [8]:
df.to_csv('../../DataSet/processed/emdat_CO_EU_JP_2023_07_05.csv', index=False)